In [1]:
import numpy as np
from scipy.special import logit,expit
import collections
import matplotlib.pyplot as plt

In [2]:
def g(x,y):
    y=np.dot(x,y)
    return y

def f(x,y):
    y=np.dot(x,y)
    return y

def pi(x,y):
    y=expit(g(x,y))
    return y

def phi(x,y):
    y=expit(f(x,y))
    return y

def I0(y):
    if y==0:
        return 1

In [19]:
#E-step
def Lih(X,Y,beta):
    Ga=np.zeros((N,2))

    for i in range(N):#响应度
        if Y[i]==1:
            Ga[i,0]=0
            Ga[i,1]=1
        elif Y[i]==0:
            a=(expit(np.dot(X[i],beta[0])))
            b=(1-a)*(1-expit(np.dot(X[i],beta[1])))
            Ga[i,0]=a/(a+b)
            Ga[i,1]=1-Ga[i,0]

    return Ga


#M-step
def Q(X,Y,beta):
    Ga=Lih(X,Y,beta)
    A=np.zeros(N)
    for i in range(N):
        pi=expit(np.dot(beta[0],X[i]))
        phi=expit(np.dot(beta[1],X[i]))
        if Y[i]==0:
            A[i]=(Ga[i,0])*np.log(pi)+(Ga[i,1])*np.log((1-pi)*(1-phi))
            #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
            #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)))
        else:
            A[i]=(Ga[i,1])*np.log((1-pi)*(phi))
            #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
            #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(phi)))
        #A[i]=(Ga[i,0]*(1-Y[i])*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)+Y[i]*np.dot(X[i],beta[1])))
    Q=np.sum(A)
    return Q

def grad(X,Y,Ga,beta):
    A=np.zeros((N,P))
    B=np.zeros((N,P))
    for i in range(N):
        A[i]=((Ga[i,0]*(1-Y[i]))*(1-expit(np.dot(beta[0],X[i])))-(Ga[i,1]*expit(np.dot(beta[0],X[i]))))*X[i]
        B[i]=(Ga[i,1]*(Y[i]-expit(np.dot(beta[1],X[i]))))*X[i]
    grad1=np.sum(A,axis=0)
    grad2=np.sum(B,axis=0)
    grad=np.array([grad1,grad2])
    return grad


#EM
def dgra(X,Y,ibeta,n=1000,t=0.001,xim=0.9,yim=0.3):#梯度下降（回溯梯度下降法）xim为步长缩小倍数

    for i in range(n):#M-step
        lik1=Q(X,Y,ibeta)
        ibeta1=ibeta
        L=Lih(X,Y,ibeta)#E-step
        ibeta=ibeta+t*grad(X,Y,L,ibeta)
        lik=Q(X,Y,ibeta)
        d=np.r_[ibeta[0],ibeta[1]]
        c=np.dot(d,d)
        b=(lik-lik1)
        print('i=',i,'b=',b,'\tstep=',t)
        print('Q=',Q(X,Y,ibeta))
        if b>=yim*t*c :
            t=xim*t
        else:
            break
    return ibeta


In [4]:
N=500
M=50
P=3
np.random.seed(123)


In [5]:
def creatB():
    b1=0.1*np.random.rand(3)-0.05
    b2=np.random.rand(3)-0.5
    b1[2]=-0.5*np.random.rand()-0.5
    b2[1]=0.5*np.random.rand()+0.5
    b=np.array([b1,b2])
    return b

def creatX():#X[i,j]表示第i个样本第j维分量
    X=np.zeros((N,3))
    for i in range(N):
        a=np.zeros(3)
        mu=[-1.5,-1.5]
        cov=np.array([[1,0.3],[0.3,1]])
        a[0]=1
        a[1:]=np.random.multivariate_normal(mu,cov,1,check_valid='raise')
        X[i]=a
    return X


def creatY(X,beta):
    Y=np.zeros(N)
    for i in range(N):
        a=np.random.rand()
        b=pi(X[i],beta[0])
        if a>=b:
            c=np.random.rand()
            d=phi(X[i],beta[1])
            if c>=d:
                Y[i]=1
    return Y

In [6]:
Y=np.zeros(N)
beta=creatB()
A=creatX()
B=expit(np.dot(A,beta.T))
C=np.random.rand(N,2)
for j in range(N):
    if C[j,0]>B[j,0]:
        if C[j,1]<B[j,1]:
            Y[j]=1

In [7]:
np.array(np.where(Y[:]==1)).shape

(1, 29)

In [8]:
X=np.zeros((M,N,3))
Y=np.zeros((M,N))
Betat=np.zeros((M,2,3))
Corr=np.zeros(M)
for i in range(M):
    beta=creatB()
    Betat[i]=beta
    A=creatX()
    X[i]=A
    B=expit(np.dot(A,beta.T))
    C=np.random.rand(N,2)
    for j in range(N):
        if C[j,0]>B[j,0]:
            if C[j,1]<B[j,1]:
                Y[i,j]=1
    Corr[i]=np.array(np.where(Y[i]==1)).shape[1]/N

In [9]:
np.mean(Betat,axis=0)

array([[-0.00372433,  0.0013206 , -0.77501074],
       [-0.02278422,  0.75427055,  0.03106094]])

In [10]:
#initial value
beta0=np.array(np.mean(Betat,axis=0))

In [18]:
hbeta=np.zeros((M,2,3))
for i in range(M):
    hbeta[i]=dgra(X[i],Y[i],beta0)


i= 0 b= 4.8911581815400496 	step= 0.001
Q= -305.0259800673356
i= 1 b= 4.116236604427797 	step= 0.00099
Q= -300.9097434629078
i= 2 b= 3.5085976757008552 	step= 0.0009801
Q= -297.40114578720693
i= 3 b= 3.021845609354216 	step= 0.000970299
Q= -294.3793001778527
i= 4 b= 2.6250758213726044 	step= 0.0009605960099999999
Q= -291.7542243564801
i= 5 b= 2.29693260164413 	step= 0.0009509900498999999
Q= -289.457291754836
i= 6 b= 2.022204045433341 	step= 0.0009414801494009999
Q= -287.43508770940264
i= 7 b= 1.7897745204497255 	step= 0.0009320653479069899
Q= -285.6453131889529
i= 8 b= 1.5913428784018038 	step= 0.00092274469442792
Q= -284.0539703105511
i= 9 b= 1.420592180270603 	step= 0.0009135172474836408
Q= -282.6333781302805
i= 10 b= 1.272635892246683 	step= 0.0009043820750088043
Q= -281.3607422380338
i= 11 b= 1.143638966983417 	step= 0.0008953382542587163
Q= -280.2171032710504
i= 12 b= 1.030552729977103 	step= 0.0008863848717161291
Q= -279.1865505410733
i= 13 b= 0.9309256913405761 	step= 0.00087752

In [21]:
np.save('hbeta.npy',hbeta)
np.save('Betat.npy',Betat)
np.save('Y.npy',Y)
np.save('X.npy',X)

In [ ]:
hbeta=np.load('hbeta.npy')
Betat=np.load('Betat.npy')
Y=np.load('Y.npy')
X=np.load('X.npy')

In [22]:
mse=hbeta-Betat

In [48]:
RMse=np.zeros(M)
for i in range(M):
    RMse[i]=np.sqrt(np.sum(mse[i]**2)/M)

In [50]:
RMse

array([0.04147613, 0.05896544, 0.09402257, 0.05350746, 0.03993648,
       0.04480065, 0.10393065, 0.0842456 , 0.05922174, 0.07543821,
       0.04305383, 0.06885363, 0.06648597, 0.05101288, 0.04054656,
       0.04731042, 0.07458137, 0.06384313, 0.04526342, 0.08154769,
       0.04783952, 0.05671296, 0.03881789, 0.07542999, 0.04891845,
       0.08093786, 0.0550198 , 0.06460779, 0.07006555, 0.091099  ,
       0.07253532, 0.0353988 , 0.0640373 , 0.05012809, 0.04192807,
       0.04712712, 0.06759402, 0.04170202, 0.03832269, 0.07336155,
       0.04639184, 0.0439663 , 0.08072333, 0.07910036, 0.04839342,
       0.09437035, 0.06462014, 0.05526628, 0.05998122, 0.03633996])

In [51]:
np.savetxt('Rmse.csv',RMse)

In [31]:
mse[0]**2

array([[4.68230553e-02, 2.92948304e-02, 1.45461021e-05],
       [2.53523071e-03, 9.03097080e-04, 6.44270206e-03]])